In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import gensim

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import string
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
!pip install torchtext==0.4

     |████████████████████████████████| 61kB 7.8MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import torch
from torchtext import data, datasets, vocab, utils
import torch.nn as nn
import torchsummary

In [0]:
PATH = '/content/drive/My Drive/Colab Notebooks/Fake_News_Detection/dataset/'

In [0]:
df = pd.read_csv(PATH+'data_clean_with_onehot.csv')

In [0]:
train_index = df.query("data_type == 'train'").index
val_index = df.query("data_type == 'val'").index
test_index = df.query("data_type == 'test'").index

In [0]:
y_train_multi = df.loc[train_index, 'label']
y_val_multi = df.loc[val_index, 'label']
y_test_multi = df.loc[test_index, 'label']

y_train_binary = df.loc[train_index, 'is_fake']
y_val_binary = df.loc[val_index, 'is_fake']
y_test_binary = df.loc[test_index, 'is_fake']

In [0]:
train_sentences = df.loc[train_index].statement.values
val_sentences = df.loc[val_index].statement.values
test_sentences = df.loc[test_index].statement.values

# Baseline

##Majority vote model (baseline)

In [0]:
def get_metrics(true, predicted, binary=True):
  if binary:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted), 4)],
  "Recall": [round(recall_score(true, predicted), 4)],
  "F1": [round(f1_score(true, predicted), 4)]})
  else:
    return pd.DataFrame({"Accuracy": [round(accuracy_score(true, predicted), 4)],
  "Precision": [round(precision_score(true, predicted, average='macro'), 4)],
  "Recall": [round(recall_score(true, predicted, average='macro'), 4)],
  "F1": [round(f1_score(true, predicted, average='macro'), 4)]})

### Multiclass classification

In [0]:
multi_popular = y_train_multi.value_counts().index[0]

In [14]:
get_metrics(y_test_multi, [multi_popular]*len(y_test_multi), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2092,0.0349,0.1667,0.0577


### Binary Classification

In [0]:
binary_popular = y_train_binary.value_counts().index[0]

In [16]:
get_metrics(y_test_binary, [binary_popular]*len(y_test_binary), binary=False)

,Accuracy,Precision,Recall,F1
0,0.5635,0.2818,0.5,0.3604


## TF-IDF LogisticRegression

### "Statement" model

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True, max_df=0.8)

In [0]:
X_train = tfidf.fit_transform(df.loc[train_index, 'statement'])
X_val = tfidf.transform(df.loc[val_index, 'statement'])
X_test = tfidf.transform(df.loc[test_index, 'statement'])

#### Multiclass classification

In [0]:
lr = LogisticRegression(max_iter=500)

In [21]:
lr.fit(X_train, y_train_multi)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
get_metrics(y_test_multi, lr.predict(X_test), binary=False)

,Accuracy,Precision,Recall,F1
0,0.2612,0.3321,0.2305,0.2232


#### Binary Classification 

In [0]:
lr = LogisticRegression(C=2)

In [24]:
lr.fit(X_train, y_train_binary)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
get_metrics(y_test_binary, lr.predict(X_test))

,Accuracy,Precision,Recall,F1
0,0.6267,0.6081,0.4069,0.4875


## TF-IDF SVM

In [0]:
svm = SVC(kernel='linear')

In [27]:
svm.fit(X_train, y_train_binary)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [28]:
get_metrics(y_test_binary, svm.predict(X_test))

,Accuracy,Precision,Recall,F1
0,0.6251,0.5995,0.425,0.4974


# GoogleNews Embeddings

In [12]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-05-31 20:51:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.14.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.14.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G   109MB/s    in 17s     

2020-05-31 20:52:02 (91.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
embeddings_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
vocab = embeddings_model.vocab
embedding = embeddings_model.syn0

In [0]:
def text_embed_info(feature):
  vocabulary = {}
  for word in ' '.join(df[feature].values).split():
    if word in vocabulary.keys():
      vocabulary[word] += 1
    else:
      vocabulary[word] = 1
  success = 0
  word_cnt = 0
  unknown_words = []
  for word in vocabulary.keys():
    if word in vocab.keys():
      success += 1
      word_cnt += vocabulary[word]
    else:
      unknown_words.append(word)

  print(f"{feature} | Found embeddings for {success*100//len(vocabulary.keys())}% of vocabulary")
  print(f"{feature} | Found embeddings for {word_cnt*100//len(' '.join(df[feature].values).split())}% of all text")
  print()

In [15]:
text_embed_info('statement')
text_embed_info('subject')
text_embed_info('speaker_job')
text_embed_info('context')
text_embed_info('justification')

statement | Found embeddings for 88% of vocabulary
statement | Found embeddings for 90% of all text

subject | Found embeddings for 98% of vocabulary
subject | Found embeddings for 98% of all text

speaker_job | Found embeddings for 92% of vocabulary
speaker_job | Found embeddings for 82% of all text

context | Found embeddings for 86% of vocabulary
context | Found embeddings for 81% of all text

justification | Found embeddings for 80% of vocabulary
justification | Found embeddings for 89% of all text



# Embeddings preprocessing

In [0]:
# Вот полная функция для фиксации сида:
# def seed_torch(seed=1029):
# random.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

In [0]:
EMBED_SIZE = 300 
BATCH_SIZE = 256

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
STATEMENT = data.Field(sequential=True)
CONTEXT = data.Field(sequential=True)
SUBJECT = data.Field(sequential=True)
SPEAKER_JOB = data.Field(sequential=True)
JUSTIFICATION = data.Field(sequential=True)

SPEAKER = data.Field(sequential=False)
STATE = data.Field(sequential=False)
PARTY = data.Field(sequential=False)

TARGET_BINAR = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)
TARGET_MULTI = data.Field(sequential=False, use_vocab=False, dtype=torch.int64)

data_fields = [('label', TARGET_MULTI),
               ('is_fake', TARGET_BINAR),
               ('statement', STATEMENT),
               ('subject', SUBJECT),
               ('speaker', SPEAKER),
               ('speaker_job', SPEAKER_JOB),
               ('state', STATE),
               ('party', PARTY),
               ('context', CONTEXT),
               ('justification', JUSTIFICATION)
               ]

In [0]:
train = data.TabularDataset(path=PATH+'train_data_clean.csv', format='csv', fields=data_fields, skip_header=True)
val = data.TabularDataset(path=PATH+'val_data_clean.csv', format='csv', fields=data_fields, skip_header=True)
test = data.TabularDataset(path=PATH+'test_data_clean.csv', format='csv', fields=data_fields, skip_header=True)

In [0]:
STATEMENT.build_vocab(train)
CONTEXT.build_vocab(train)
SUBJECT.build_vocab(train)
SPEAKER_JOB.build_vocab(train)
JUSTIFICATION.build_vocab(train)
SPEAKER.build_vocab(train)
STATE.build_vocab(train)
PARTY.build_vocab(train)

In [21]:
statement_vocab_size = len(STATEMENT.vocab.itos)
context_vocab_size = len(CONTEXT.vocab.itos)
subject_vocab_size = len(SUBJECT.vocab.itos)
speaker_vocab_size = len(SPEAKER.vocab.itos)
speaker_job_vocab_size = len(SPEAKER_JOB.vocab.itos)
state_vocab_size = len(STATE.vocab.itos)
party_vocab_size = len(PARTY.vocab.itos)
justification_vocab_size = len(JUSTIFICATION.vocab.itos)

print(f"Statement | Vocabulary size {statement_vocab_size}")
print(f"Context | Vocabulary size {context_vocab_size}")
print(f"Subject | Vocabulary size {subject_vocab_size}")
print(f"Speaker | Vocabulary size {speaker_vocab_size}")
print(f"Speaker_job | Vocabulary size {speaker_job_vocab_size}")
print(f"State | Vocabulary size {state_vocab_size}")
print(f"Party | Vocabulary size {party_vocab_size}")
print(f"Justification | Vocabulary size {justification_vocab_size}")

Statement | Vocabulary size 11592
Context | Vocabulary size 2153
Subject | Vocabulary size 178
Speaker | Vocabulary size 162
Speaker_job | Vocabulary size 1102
State | Vocabulary size 60
Party | Vocabulary size 24
Justification | Vocabulary size 22266


In [34]:
len(STATEMENT.vocab)

11592

In [0]:
def get_embedding_matrix(feature):
  emb_mean, emb_std = embedding.mean(), embedding.std()

  word_index = dict(feature.vocab.stoi).keys()
  unknown_words = set()
  nb_words = len(word_index)
  embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
  for word in word_index:
        index = feature.vocab[word]
        if index != 0:
          try:
            embedding_vector = embeddings_model[word]
            embedding_matrix[index] = embedding_vector
          except KeyError:
            unknown_words.add(word)
        else:
          continue

  embedding_matrix = torch.from_numpy(embedding_matrix)
  print(f"Unknown words {len(unknown_words)}")

  return embedding_matrix, unknown_words

In [0]:
def get_one_hot_matrix(feature):
  embedding_matrix = torch.from_numpy(np.eye(len(feature.vocab)))
  return embedding_matrix

In [24]:
statement_embedding_matrix, statement_unknown_words = get_embedding_matrix(STATEMENT)

Unknown words 1243


In [25]:
context_embedding_matrix, context_unknown_words = get_embedding_matrix(CONTEXT)

Unknown words 280


In [26]:
subject_embedding_matrix, subject_unknown_words = get_embedding_matrix(SUBJECT)

Unknown words 3


In [27]:
speaker_job_embedding_matrix, speaker_job_unknown_words = get_embedding_matrix(SPEAKER_JOB)

Unknown words 82


In [28]:
justification_embedding_matrix, justification_unknown_words = get_embedding_matrix(JUSTIFICATION)

Unknown words 4024


In [0]:
speaker_embedding_matrix = get_one_hot_matrix(SPEAKER)
state_embedding_matrix = get_one_hot_matrix(STATE)
party_embedding_matrix = get_one_hot_matrix(PARTY)

In [0]:
STATEMENT.vocab.set_vectors(STATEMENT.vocab.stoi, statement_embedding_matrix, EMBED_SIZE)
CONTEXT.vocab.set_vectors(CONTEXT.vocab.stoi, context_embedding_matrix, EMBED_SIZE)
SUBJECT.vocab.set_vectors(SUBJECT.vocab.stoi, subject_embedding_matrix, EMBED_SIZE)
SPEAKER_JOB.vocab.set_vectors(SPEAKER_JOB.vocab.stoi, speaker_job_embedding_matrix, EMBED_SIZE)
JUSTIFICATION.vocab.set_vectors(JUSTIFICATION.vocab.stoi, justification_embedding_matrix, EMBED_SIZE)

In [0]:
SPEAKER.vocab.set_vectors(SPEAKER.vocab.stoi, speaker_embedding_matrix, speaker_vocab_size)
STATE.vocab.set_vectors(STATE.vocab.stoi, state_embedding_matrix, state_vocab_size)
PARTY.vocab.set_vectors(PARTY.vocab.stoi, party_embedding_matrix, party_vocab_size)

In [0]:
train_iter = data.BucketIterator(train,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                shuffle=True,
                                device=device)

val_iter = data.BucketIterator(val,
                                batch_size=BATCH_SIZE,
                                sort_key= lambda x: len(x.statement),
                                repeat=False,
                                sort = True,
                                device=device)

## Bi-LSTM (Statement)

In [0]:
output_size = 2
hidden_size = 300

In [0]:
class BILSTMClassifier(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, embedding_length):
        super(BILSTMClassifier, self).__init__()

        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 3 = (normal, small, big)
        hidden_sie : Size of the hidden_state of the LSTM
        embedding_length : Embeddding dimension of FastText word embeddings
        
        """

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.embedding_length = embedding_length
        self.embedding_length_speaker = len(SPEAKER.vocab)
        self.embedding_length_state = len(STATE.vocab)
        self.embedding_length_party = len(PARTY.vocab)

        self.embedding_statement = nn.Embedding(len(STATEMENT.vocab), embedding_length, padding_idx=1)
        self.embedding_statement.weight = nn.Parameter(STATEMENT.vocab.vectors, requires_grad=True)

        self.embedding_context = nn.Embedding(len(CONTEXT.vocab), embedding_length, padding_idx=1)
        self.embedding_context.weight = nn.Parameter(CONTEXT.vocab.vectors, requires_grad=True)

        self.embedding_subject = nn.Embedding(len(SUBJECT.vocab), embedding_length, padding_idx=1)
        self.embedding_subject.weight = nn.Parameter(SUBJECT.vocab.vectors, requires_grad=True)

        self.embedding_speaker_job = nn.Embedding(len(SPEAKER_JOB.vocab), embedding_length, padding_idx=1)
        self.embedding_speaker_job.weight = nn.Parameter(SPEAKER_JOB.vocab.vectors, requires_grad=True)

        self.embedding_justification = nn.Embedding(len(JUSTIFICATION.vocab), embedding_length, padding_idx=1)
        self.embedding_justification.weight = nn.Parameter(JUSTIFICATION.vocab.vectors, requires_grad=True)

        self.embedding_speaker = nn.Embedding(len(SPEAKER.vocab), len(SPEAKER.vocab), padding_idx=1)
        self.embedding_speaker.weight = nn.Parameter(SPEAKER.vocab.vectors, requires_grad=False)

        self.embedding_state = nn.Embedding(len(STATE.vocab), len(STATE.vocab), padding_idx=1)
        self.embedding_state.weight = nn.Parameter(STATE.vocab.vectors, requires_grad=False)

        self.embedding_party = nn.Embedding(len(PARTY.vocab), len(PARTY.vocab), padding_idx=1)
        self.embedding_party.weight = nn.Parameter(PARTY.vocab.vectors, requires_grad=False)



        self.lstm_statement = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=1)
        self.bn1 = nn.BatchNorm1d(hidden_size*2)
        self.relu1 = nn.ReLU()

        self.lstm_context = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=1)
        self.bn2 = nn.BatchNorm1d(hidden_size*2)
        self.relu2 = nn.ReLU()

        self.lstm_subject = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=1)
        self.bn3 = nn.BatchNorm1d(hidden_size*2)
        self.relu3 = nn.ReLU()

        self.lstm_speaker_job = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=1)
        self.bn4 = nn.BatchNorm1d(hidden_size*2)
        self.relu4 = nn.ReLU()

        self.lstm_justification = nn.LSTM(embedding_length, hidden_size, bidirectional=True, num_layers=1)
        self.bn5 = nn.BatchNorm1d(hidden_size*2)
        self.relu5 = nn.ReLU()

        self.fnn_categorical = nn.Linear(self.embedding_length_speaker +
                                         self.embedding_length_party +
                                         self.embedding_length_state, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.relu6 = nn.ReLU()
        

        self.fnn_lstm = nn.Linear(hidden_size*2*5, 2048)
        self.bn7 = nn.BatchNorm1d(2048)
        self.relu7 = nn.ReLU()

        self.label = nn.Linear(2560, output_size)

    def forward(self, statement, subject, speaker, speaker_job, state, party, context, justification, batch_size=None):
        statement_embed = self.embedding_statement(statement)
        context_embed = self.embedding_context(context)
        subject_embed = self.embedding_subject(subject)
        speaker_job_embed = self.embedding_speaker_job(speaker_job)
        justification_embed = self.embedding_justification(justification)
        state_embed = self.embedding_state(state)
        speaker_embed = self.embedding_speaker(speaker)
        party_embed = self.embedding_party(party)

        _, (statement_final_hidden_state, _) = self.lstm_statement(statement_embed)
        _, (context_final_hidden_state, _) = self.lstm_context(context_embed)
        _, (subject_final_hidden_state, _) = self.lstm_subject(subject_embed)
        _, (speaker_job_final_hidden_state, _) = self.lstm_speaker_job(speaker_job_embed)
        _, (justification_final_hidden_state, _) = self.lstm_justification(justification_embed)

        statement_final_hidden_state = torch.cat([statement_final_hidden_state[i,:,:] for i in range(int(statement_final_hidden_state.shape[0]))], dim=1)
        context_final_hidden_state = torch.cat([context_final_hidden_state[i,:,:] for i in range(int(context_final_hidden_state.shape[0]))], dim=1)
        subject_final_hidden_state = torch.cat([subject_final_hidden_state[i,:,:] for i in range(int(subject_final_hidden_state.shape[0]))], dim=1)
        speaker_job_final_hidden_state = torch.cat([speaker_job_final_hidden_state[i,:,:] for i in range(int(speaker_job_final_hidden_state.shape[0]))], dim=1)
        justification_final_hidden_state = torch.cat([justification_final_hidden_state[i,:,:] for i in range(int(justification_final_hidden_state.shape[0]))], dim=1)

        statement_final_hidden_state = self.relu1(self.bn1(statement_final_hidden_state))
        context_final_hidden_state = self.relu2(self.bn2(context_final_hidden_state))
        subject_final_hidden_state = self.relu3(self.bn3(subject_final_hidden_state))
        speaker_job_final_hidden_state = self.relu4(self.bn4(speaker_job_final_hidden_state))
        justification_final_hidden_state = self.relu5(self.bn5(justification_final_hidden_state))

        fnn_categorical = self.relu6(self.bn6(self.fnn_categorical(torch.cat([state_embed, speaker_embed, party_embed], dim=1))))

        fnn_lstm = self.relu7(self.bn7(self.fnn_lstm(torch.cat([statement_final_hidden_state, context_final_hidden_state,
                                            subject_final_hidden_state, speaker_job_final_hidden_state,
                                            justification_final_hidden_state], dim=1))))
        

        final_output = self.label(torch.cat([fnn_categorical, fnn_lstm], dim=1))
        return final_output

In [0]:
lstm_model = BILSTMClassifier(BATCH_SIZE, output_size, hidden_size, EMBED_SIZE).to(device)

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(lstm_model.parameters())
loss_func = nn.CrossEntropyLoss().to(device)

In [71]:
for epoch in range(5): 

    running_loss = 0
    train_acc = 0

    for el in train_iter:
        labels = el.is_fake.to(device)
        input_statement = el.statement.to(device)
        input_subject = el.subject.to(device)
        input_speaker = el.speaker.to(device)
        input_speaker_job = el.speaker_job.to(device)
        input_state = el.state.to(device)
        input_party = el.party.to(device)
        input_context = el.context.to(device)
        input_justification = el.justification.to(device)
      
        optimizer.zero_grad()

        outputs = lstm_model(input_statement, input_subject, input_speaker, input_speaker_job, input_state, input_party, input_context, input_justification)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        train_acc += (outputs.argmax(1) == labels).sum().item()
        running_loss += loss.item()
    
    val_loss = 0
    val_acc = 0

    for el in val_iter:
      labels = el.is_fake.to(device)
      input_statement = el.statement.to(device)
      input_subject = el.subject.to(device)
      input_speaker = el.speaker.to(device)
      input_speaker_job = el.speaker_job.to(device)
      input_state = el.state.to(device)
      input_party = el.party.to(device)
      input_context = el.context.to(device)
      input_justification = el.justification.to(device)

      outputs = lstm_model(input_statement, input_subject, input_speaker, input_speaker_job, input_state, input_party, input_context, input_justification)
      loss = loss_func(outputs, labels)
      val_acc += (outputs.argmax(1) == labels).sum().item()
      val_loss += loss.item()

    train_loss = running_loss / (len(train) / BATCH_SIZE)
    train_acc = train_acc / len(train)
    val_loss = val_loss / (len(val) / BATCH_SIZE)
    val_acc = val_acc / len(val)

    print(f'Epoch: {epoch + 1}')
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {val_loss:.4f}(valid)\t|\tAcc: {val_acc * 100:.1f}%(valid)')


print('Finished Training')

Epoch: 1
	Loss: 0.7689(train)	|	Acc: 59.7%(train)
	Loss: 0.7378(valid)	|	Acc: 61.8%(valid)
Epoch: 2
	Loss: 0.5807(train)	|	Acc: 69.0%(train)
	Loss: 0.8845(valid)	|	Acc: 62.6%(valid)
Epoch: 3
	Loss: 0.4437(train)	|	Acc: 79.9%(train)
	Loss: 1.0008(valid)	|	Acc: 63.2%(valid)
Epoch: 4
	Loss: 0.2346(train)	|	Acc: 90.8%(train)
	Loss: 1.8694(valid)	|	Acc: 58.2%(valid)


KeyboardInterrupt: ignored